In [85]:
import pandas as pd 
import pickle
import geopandas as gpd

import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings("ignore")

In [86]:
contour_iris=gpd.read_file(r'Y:\REFERENTIEL DATA\RP 2022 FRANCE\CARTOGRAPHIE\Cartographie_France_IRIS_2019\CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019\CONTOURS-IRIS.shp').set_crs("EPSG:2154")

In [87]:
list_departement=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Liste-Excel-des-departements-francais.xlsx")


In [88]:
logs=pickle.load(open(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\\logs.pkl","rb"))

In [90]:
for i ,row in list_departement.iterrows():

 
        try:
            dep=int(row['N°'])
            gsa_dep=gsa[gsa['CP'].isin([i for i in range(dep*1000,(dep+1)*1000) ])]
            gss_dep=gss[gss['CP'].isin([i for i in range(dep*1000,(dep+1)*1000) ])]
        except:
            gsa_dep=gsa.copy()
            gss_dep=gss.copy()
        place_name=row["Département"].lower().strip()
        try:
            path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\\"+row['Région'].lower()+r"\\"+row["Département"].lower()

            buildings=pd.read_pickle(path+r"\\processed data\\buildings.pkl").to_crs("2154")

            #########################################################################
            #magasins /GSA/GSS

            shops=pd.read_pickle(path+r"\\raw data\\shops.pkl").to_crs("2154")
            a=gpd.sjoin(buildings,shops,predicate="contains",lsuffix='left', rsuffix='right').to_crs('EPSG:4326')
            pickle.dump(a,open(path+r"\\association\\buildings_shops.pkl",'wb'))
            #index right : la position de gss/gsa dans la table gsa/gss en dessous
            try:
                gss=pickle.load(open(path+"\\processed data\\gss_comp.pkl","rb")).to_crs("2154")
                b_gss=gpd.sjoin(buildings,gss,predicate="contains")[['geometry',"index_right"]]
                pickle.dump(b_gss,open(pathe+r"\\association\\buildings_gss.pkl",'wb'))
            except:pass

            try:
                gsa=pickle.load(open(path+"\\processed data\\gsa_comp.pkl","rb")).to_crs("2154")
                b_gsa=gpd.sjoin(buildings,gsa,predicate="contains")[['geometry',"index_right"]]
                pickle.dump(b_gsa,open(path+r"\\association\\buildings_gsa.pkl",'wb'))
            except:pass
            #logs
            logs[place_name].update({"buildings_shops":len(a),"buildings_gss":len(b_gss),"buildings_gsa":len(b_gsa)})

            ##########################################################################
            #tourisme
            tourism=pd.read_pickle(path+r"\\processed data\\tourism.pkl").to_crs("2154")
            a=gpd.sjoin(buildings,tourism,predicate="contains").to_crs('EPSG:4326')[['geometry',"index_right0","index_right1"]]
            pickle.dump(a,open(path+r"\\association\\buildings_tourism.pkl",'wb'))

            #logs
            logs[place_name].update({"buildings_tourism":len(a)})

            ##############################################################################
            #loisire
            leisure=pd.read_pickle(path+r"\\raw data\\leisure.pkl").to_crs("2154")
            a=gpd.sjoin(buildings,leisure,predicate="contains").to_crs('EPSG:4326')[['geometry',"index_right0","index_right1"]]
            pickle.dump(a,open(path+r"\\association\\buildings_leisure.pkl",'wb'))

            #logs
            logs[place_name].update({"buildings_leisure":len(a)})

            ##############################################################################
            #amenity
            amenity=pd.read_pickle(path+r"\\processed data\\amenity.pkl").to_crs("2154")
            a=gpd.sjoin(buildings,amenity,predicate="contains").to_crs('EPSG:4326')[['geometry',"index_right0","index_right1"]]
            pickle.dump(a,open(path+r"\\association\\buildings_amenity.pkl",'wb'))

            #logs
            logs[place_name].update({"buildings_amenity":len(a)})
        except:
            print(place_name)

hautes-pyrénées
pyrénées-orientales
bas-rhin
haut-rhin
haute-savoie
territoire-de-belfort
guadeloupe
martinique
guyane
la réunion
mayotte
